In [1]:
import tensorflow as tf
# from beholder import Beholder
from tensorboard.plugins.beholder import Beholder
beholder = Beholder("output")

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.test.images.dtype

dtype('float32')

In [4]:
hidden_1 = 500
hidden_2 = 500
hidden_3 = 500

n_classes = 10
epochs = 10
batch_size = 128
lr = 1e-3 
tf.reset_default_graph()

In [5]:
with tf.name_scope('inputs'):
    x = tf.placeholder('float', [None, 28*28])
    y = tf.placeholder('float', [None, 10])
acc_ = tf.placeholder('float')
acc_val_ = tf.placeholder('float')
c = tf.Variable('float')

In [6]:
dropout = 0.4
def neural_network():
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    with tf.name_scope('reshape'):
        img = tf.reshape(x, shape=[-1, 28, 28, 1])
        
        # Convolution Layer with 32 filters and a kernel size of 5
    with tf.name_scope('convolution_1'):
        conv1 = tf.layers.conv2d(img, 32, 5, 
                                 activation=tf.nn.relu,
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        max_pool1 = tf.layers.max_pooling2d(conv1, 2, 2)
#         variable_summaries(max_pool1)
        # Convolution Layer with 64 filters and a kernel size of 3
    with tf.name_scope('convolution_2'):    
        conv2 = tf.layers.conv2d(max_pool1, 64, 3, 
                                 activation=tf.nn.relu,
                                 kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))
        max_pool2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
    with tf.name_scope('flatten_dense'):
        fc1 = tf.contrib.layers.flatten(conv2)
        fc1 = tf.layers.dense(fc1, 1024, 
                              activation = tf.nn.relu,
                             kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

        # Output layer, class prediction
    with tf.name_scope('outputs'):
        output = tf.layers.dense(fc1, n_classes, activation = tf.nn.softmax,
                                kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

    return output

In [8]:
prediction = neural_network()
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction, labels = y))
optimizer = tf.train.AdamOptimizer(lr, name = "Adam").minimize(cost)

correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

epochs = 2

accuracy_summary = tf.summary.scalar('Accuracy', acc_)
accuracy_val_summary = tf.summary.scalar('Accuracy_Validation', acc_val_)

saver = tf.train.Saver()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        avg_loss =  0
        for _ in range(int(mnist.train.num_examples/batch_size)):
            x_epoch, y_epoch = mnist.train.next_batch(batch_size)
            
            _, c = sess.run([optimizer, cost], feed_dict = {x:x_epoch, y:y_epoch})
            avg_loss += c / int(mnist.train.num_examples/batch_size)
        print('Epoch', epoch, 'completed out of', epochs, 'loss ', avg_loss)       
    
    save_path = saver.save(sess, "model/model.ckpt")
    print("Model saved in path: %s" % save_path)  

#     gr = tf.get_default_graph()
#     conv1_kernel_val = gr.get_tensor_by_name('conv2d_2/kernel:0').eval(session = sess)

Epoch 0 completed out of 2 loss  1.527210428442432
Epoch 1 completed out of 2 loss  1.481690270083768
Model saved in path: model/model.ckpt


In [21]:
config = tf.ConfigProto(
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "model/model.ckpt")
    gr = tf.get_default_graph()
#     conv1_kernel_val_1 = gr.get_tensor_by_name('conv2d_2/kernel:0').eval(session = sess)
#     conv1_kernel_val_2 = gr.get_tensor_by_name('conv2d_3/kernel:0').eval(session = sess)
    pred = sess.run(prediction, feed_dict = {x:[mnist.test.images[0]]})

INFO:tensorflow:Restoring parameters from model/model.ckpt


In [22]:
pred.shape


(1, 10)

In [ ]:
# conv1_kernel_val_1.shape# Convolution Layer with 32 filters and a kernel size of 5

In [ ]:
# conv1_kernel_val_2.shape# Convolution Layer with 64 filters and a kernel size of 3

In [ ]:
# tmep = conv1_kernel_val_1.reshape(-1)
# tmep = tmep.reshape(32, 25)

In [ ]:
# import matplotlib.pyplot as plt
# for i in range(32):
#     plt.imshow(tmep[i].reshape(5, 5), cmap = 'gray')
#     plt.show()

In [ ]:
import numpy as np
predicted_values = []
for i in range(len(pred[0])):
    index_max = np.argmax(pred[0][i])
    predicted_values.append(index_max)

In [ ]:
actual = []
for i in range(len(pred[0])):
    index_max = np.argmax(mnist.test.labels[i])
    actual.append(index_max )

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
 # unnormalize
    #npimg = img.numpy()
    plt.imshow(np.transpose(img[0], (1, 2, 0)))
    plt.show()
plt.imshow((mnist.test.images[0]).reshape((28, 28)))

In [ ]:
def classification_metrics(actual, pred, msg):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(actual, pred)
    import seaborn as sns     

    plt.figure()
    ax= plt.subplot()
    sns.heatmap(cm, annot = True, fmt = 'g')

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']) 
    ax.yaxis.set_ticklabels(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])   
    plt.show()       
    sensitivity = cm[1][1]/(cm[1][0] + cm[1][1])
    specifity = cm[0][0]/(cm[0][0] + cm[0][1])
    accuracy = (cm[0][0] + cm[1][1]) / np.sum(cm)
    
#     print(msg, '\n')
#     print('accuracy:    ', round(accuracy,2), 
#       '\nsensitivity: ', round(sensitivity,2), 
#       '\nspecifity:   ', round(specifity,2))
    from sklearn.metrics import classification_report
    print(classification_report(actual, pred))
              


In [ ]:
classification_metrics(actual, predicted_values, msg = '')

## tensorboard --logdir=output/ --host localhost --port 8088